#### Modified version

In [ ]:
import fitz # pip install PyMuPDF
from datetime import date, timedelta
import html

TEMPLATE = "Weekly Newsletter Template v3.pdf"
doc      = fitz.open(TEMPLATE)

# ──────────────────── derive title & output
today    = date.today()
week_no  = int((today - timedelta(days=today.weekday())).strftime("%V"))
week_no = 17
week_tag    = f"Week {week_no}, {today.year}"
OUTPUT   = f"{today.year % 100}w{week_no:02d}.pdf"

# ──────────────────── pick up real font name (not alias!)
font_tag = None
fid, ftype, fsubtype, fname, tag, enc = doc[0].get_fonts()[0]
if "KaiseiTokumin" in fname:
	font_tag = tag         # e.g. 'F9'
	font_name = fname #Default font_name = "Helvetica"
assert font_tag, "Kaisei Tokumin not found!"

# ──────────────────── rectangles & payload
anchors = {
    "title":        fitz.Rect(12,  12, 588, 208),
    "events":       fitz.Rect(15, 252, 365, 428),
    "gratitude":    fitz.Rect(15, 448, 275, 528), #y0 decreased to accommodate bullet
    "productivity": fitz.Rect(15, 543, 275, 623), #y0 decreased to accommodate bullet
    "up_next":      fitz.Rect(15, 665, 340, 805),
    "facts":        fitz.Rect(290, 465, 585, 625),
    "img_rect":     fitz.Rect(375, 220, 585, 430),
    "weekly": 		fitz.Rect(359, 660, 464, 795),
}

payload = {
"title":        
    "A Fresh Start in Information Era",
"events":       
	"""
 	<span style="font-size:12pt;font-weight:bold">Infrastructure on Info Acquisition</span>
 	<ul style = "margin-left: 0; padding-left: 1em;">
		<li><span style="color:#FF6347;">Kick-started Q2 Interface Integration</span> and outlined project blocks for next few weeks.</li>
		<li>Python: 
  			<a href=https://tsnbowlingtracker.streamlit.app/>Bowling dashboard</a>
     		+ Newsletter repo</li>
		<li>RL (CS234) Lectures 4, 6, 7; Tue met professor</li>
		<li>Dalio "The Fund" Ch 4</li>
	</ul>
 	<span style="font-size:12pt;font-weight:bold">Personal</span>
  	<ul style = "margin-left: 0; padding-left: 1em; padding-top: 0;">
		<li>Lunch w/ TWGSS junior + cycling w/ QFIN junior</li>
		<li>Significant practice sessions @ 
  			<a href=https://cjdgrevival.com/>Taiko no Tatsujin</a>
     	</li>
	</ul>
	""",
"gratitude":
	"""
	<ul style = "margin-left: 0; padding-left: 1em; padding-top: 0;">
		<li>
  			<span style="color:#FF6347;">Call with friend</span>: 
     		it warms me knowing that we both anticipate for the call</li>
		<li>Peaceful closures of prev. relationships</li>
  		<li>Leveraging LLM expansion inspires me</li>
	</ul>
 	""",
"productivity": 
    """
    <ul style = "margin-left: 0; padding-left: 1em; padding-top: 0;">
		<li>YT content summary shortcut: 
  			<a href=https://downsub.com/>Downsub</a>
    	</li>
		<li>Extract and annotate PDF via
			<a href=https://pymupdf.readthedocs.io/en/latest/index.html>PyMuPDF</a>
  		</li>
		<li>
			<span style="font-size:12pt;font-weight:bold">Study</span>:
			<span style="color:red;">red (attention)</span> 
			and
			<span style="color:blue;">blue (creativity)</span> 
			colors around  
			<a href=https://pmc.ncbi.nlm.nih.gov/articles/PMC4880552/>improve brain performance</a>
		</li>
	</ul>
    """,
"up_next":      
	"""<span style="font-size:12pt;font-weight:bold">Next week: Code & App roll-outs</span>
	<ul style = "margin-left: 0; padding-left: 1em;">
		<li>
  			<span style="color:#FF6347;">Automate YT Insights</span>
     		via 
			<a href=https://pypi.org/project/youtube-transcript-api/>Py API library</a>
       	</li>
		<li>Trade Strategy Expansion/ Testing on MDP (diff. horizons)</li>
		<li>Document Lec4 CS234, study Lec 7-8 (Wrap up PPO)</li>
		<li>
  			<span style="color:#FF6347;">Revitalize DSE M2 2022 reminders deck</span> 
  			for 2025: <br> brokered deal w/ 5 sec sch teachers
    	</li>
	</ul>
	<span style="font-size:12pt;font-weight:bold">Longer Term</span>:
	Website v4, Timetable matcher, Fina. data/ sentiment handler, Read RL on LLM areas, 10GraphsPW
 	""",
"facts":
	"""<span style="font-size:12pt;font-weight:bold">Far-from-ending Market Tumble</span>
 	<ul style = "margin-left: 0; padding-left: 1em; padding-top: 0;">
		<li>GS: Retail equities net buy @ record-breaking pace</li>
		<li>
  			<span style="color:#FF6347;">6.5tn debt due in Jun'25</span>
     		, JPow hawkish resistance to Trump spells crisis: 30Y bond flirted 5%
       	</li>
		<li>Business expected to refinance at high rates shortly (5Y from COVID)</li>
  		<li>Housing defaults @ Freddie Mac stayed 0.4% over 2024-25, broke 2008 records</li>
	</ul>	
 	""",
"img_rect":     
    "Test_image",
"weekly":
    """<a href=https://www.instagram.com/p/DIfGXKNBjEz/>#98: Tariff Poker</a>
    discusses economic rationale of tariffs, the strategic behavior of nations, 
    and stakeholder insights from game theory.
    """
}

title = payload["title"]
payload["title"] = f"{week_tag}: {title}"
def make_html(src, is_title=False):
    if src.lstrip().startswith("<"):
        return src
    if is_title:
        return f'<h1 style="font-family:\'{font_name}\';margin:0">{html.escape(src)}</h1>'

    lines = src.splitlines()
    if all(l.startswith(("•", "➜", "-")) for l in lines if l.strip()):
        lis = "".join(f"<li>{html.escape(l.lstrip('•➜- '))}</li>"
                      for l in lines if l.strip())
        body = f"<ul>{lis}</ul>"
    else:
        body = "<br>".join(html.escape(l) for l in lines)

    return (f'<div style="font-family:\'{font_name}\';font-size:11pt;vertical-align:top;'
            f'line-height:13pt">{body}</div>')

# ──────────────────── draw on page 0
page = doc[0]
for key, rect in anchors.items():
    #if key == "img_rect":
    #    continue
    html_snip = make_html(payload[key], is_title=(key == "title"))
    try:
        # page.draw_rect(rect, color=(0, 0, 1), width=0.5) #Debug by displaying boundaries
        status = page.insert_htmlbox(rect, html_snip, scale_low=0.5, overlay=True)
        spare, scale = status
        if spare < 0:
            print(f"⚠️ '{key}' did not fully fit. scale={scale:.2f}, extra height needed: {-spare:.2f}pt")
    except OverflowError as e:
        print(f"🚨 OverflowError when inserting '{key}':", e)

# ───────── banner image + hyperlink
page.insert_image(anchors["img_rect"], filename=f"{today.year % 100}w{week_no:02d}-image.png",
                  keep_proportion=True, overlay=True)
#page.add_link(anchors["img_rect"],
#              uri="https://github.com/your‑repo/newsletter",
#              kind=fitz.LINK_URI)

doc.save(OUTPUT, deflate=True, garbage=4)
print("✅  Saved", OUTPUT)

✅  Saved 25w16.pdf


#### 1a) Auto-mail to subscribers from a CSV

Caveat: smtp app password is required on google!

In [10]:
import smtplib
import ssl
import pandas as pd
from email.message import EmailMessage

email = "theskillfulnoob2002@gmail.com"
#pw = "the-SKILLFUL-2002"
app_pw = "wyxl fwfv vqpr udde"
def send_newsletter(csv_path, subject, plain_body, html_body, attachment_path=None, debug_email = None):
    if debug_email or not csv_path:
        bcc_list = [debug_email]
    else:
        df = pd.read_csv(csv_path)
        bcc_list = df['email'].tolist()
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = email
    msg['To'] = email  # visible To field (yourself or generic newsletter identity)
    msg['Bcc'] = ', '.join(bcc_list)
    msg.set_content(plain_body)
    msg.add_alternative(html_body, subtype='html')

    if attachment_path:
        with open(attachment_path, 'rb') as f:
            msg.add_attachment(f.read(), maintype='application', subtype='pdf',
                               filename=attachment_path)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email, app_pw)
        smtp.send_message(msg)

#### 1b) HTML Format Editor

In [ ]:
html_content = f"""
<html>
  <body style="font-family:Arial, sans-serif; margin:0; padding:0;">
    <div style="background-color:#E0F7FA; padding:24px;">
      <div style="margin-top:20px;">
		<div style="background-color:#FFFACD; padding:16px; border-radius:8px;">
			<h2 style="color:#4CAF50; margin-top:0;">Your Weekly Digest</h2>
			<p>Hi there! ☀️</p>
			<p>Much gratitude for being my <b>first-week reader</b> 🥳 for the automated newsletter production! 
			<p>Your full newsletter attached as a PDF! Major summary below:</p>
		</div>
  
        <h4>Professional</h4>
			<ol>
			<li>Kickstarted my Q2 Interface Integration Plan: 
   			Completed Newsletter automation and <a href="https://tsnbowlingtracker.streamlit.app/">Streamlit bowling dashboard</a> 
			this week.
			</li>
			<li>Fun fact study:
				<span style="color:red;">red (attention)</span>
				and
				<span style="color:blue;">blue (creativity)</span>
				colors around
				<a href="https://pmc.ncbi.nlm.nih.gov/articles/PMC4880552/">improve brain performance</a>
			</li>
			<li>Underweight US equity! Trump would struggle to handle 6.5tn debt due Jun'25.
				I think gold (and to a lesser extent, BTC) are good counters. Some said JPY but it's prone to an earthquake.
			</li>
			</ol>
   
		<h4>Personal</h4>
			<ol>
			<li>This week opting for a special HKDSE M2 reminder deck production - a bit philanthropic (I was in the past), but I've brokered a deal to collaborate with teachers!</li>
			<li>Feeling with more energy and sense of achievement as I re-concentrated on work and materialized restructuring plans.</li>
			</ol>
        
        <p style="font-style:italic; color: #3994cc;">Stay inspired and informed.
			<br>Feel free to <b>give me feedback</b> from time to time - love to hear from you too! ❤️
			<br>- Andrew
		</p>
      </div>
    </div>

    <div style="background-color:#F0F0F0; padding:16px; font-size:0.9em; color:#555; text-align:center;">
      You received this email because you're subscribed to Andrew's Weekly Newsletter.
      <br>
      You can <a href="mailto:{email}">Unsubscribe</a> by emailing me.
    </div>
  </body>
</html>
"""

## TOGGLE SWITCH

In [ ]:
subject = "🌱 2025 W16 Newsletter: A Fresh Start in the Info Era"
attachment = "Published/25w16.pdf"
csv_path = "recipients.csv"
plain_text = "Your Week 16 newsletter is attached. If you cannot view the HTML version, please check the PDF."
send_newsletter(#csv_path=csv_path, #uncomment this for production
                csv_path = None, #comment this for production
                subject=subject, 
                plain_body=plain_text, 
                html_body=html_content, 
                attachment_path=attachment,
                debug_email = email, #comment this for production
                )

#### 1c) Add and Drop Subscribers

In [16]:
from datetime import date

def modify_subscription(email, csv_path, action='add'):
    df = pd.read_csv(csv_path)
    if action == 'add':
        if email not in df['email'].values:
            new_entry = pd.DataFrame([[email, date.today().isoformat()]], columns=['email', 'subscribed_date'])
            df = pd.concat([df, new_entry], ignore_index=True)
            print(f"Added {email} to the list!")
        else:
            print(f"{email} already exists on the list!")
    elif action == 'drop':
        df = df[df['email'] != email]
    df.to_csv(csv_path, index=False)

for add in ['janiceyeung0225@gmail.com', 'sflamad@connect.ust.hk']:
    modify_subscription(add, csv_path, 'add')

Added janiceyeung0225@gmail.com to the list!
Added sflamad@connect.ust.hk to the list!
